# CNN

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime # datetime.now()를 이용하여 학습 경과 시간 측정

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

print("\n", mnist.train.num_examples, mnist.test.num_examples, mnist.validation.num_examples)

print("\ntrain image shape = ", np.shape(mnist.train.images))
print("train label shape = ", np.shape(mnist.train.labels))
print("test image shape = ", np.shape(mnist.test.images))
print("test label shape = ", np.shape(mnist.test.labels))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz

 55000 10000 5000

train image shape =  (55000, 784)
train label shape =  (55000, 10)
test image shape =  (10000, 784)
test label shape =  (10000, 10)


In [3]:
# Hyper-Parameter
learning_rate = 0.001
epochs = 30
batch_size = 100 # 한번에 입력으로 주어지는 MNIST 개수

In [4]:
# 입력과 정답을 위한 플레이스 홀더 정의
X = tf.placeholder(tf.float32, [None, 784])
T = tf.placeholder(tf.float32,[None, 10])

# 입력층의 출력값, 컨볼루션 연산을 위해 reshape 시킴
A1 = X_img = tf.reshape(X, [-1,28,28,1]) # image 28x28x1(black/white)

In [5]:
# 1번째 컨볼루션 층
# 3x3 크기를 가지는 32개의 필터를 적용

F2 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01)) # 입력채널(데이터가 들어오는 통로가 1개, 32개 필터)
                                                            # 즉 A1은 1개의 통로를 통해서 들어옴
                                                            # 3x3의 총 32개 필터가 표준편차 0.01을 갖도록 임의의 값으로 초기화
b2 = tf.Variable(tf.constant(0.1, shape = [32])) # 필터가 32개이므로 # constant 정의

# 1번째 컨볼루션 연산을 통해 28x28x1 => 28x28x32(필터수)
C2 = tf.nn.conv2d(A1, F2, strides=[1,1,1,1], padding = 'SAME')

# relu
Z2 = tf.nn.relu(C2+b2)

# 1번째 max pooling을 통해 28x28x32 => 14x14x32
A2 = P2 = tf.nn.max_pool(Z2, ksize = [1,2,2,1], strides=[1,2,2,1], padding='SAME') # 데이터가 부족하면 padding해라

In [6]:
# 2번째 컨볼루션 층
# 3x3 크기를 가지는 64개의 필터를 적용
F3 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
b3 = tf.Variable(tf.constant(0.1, shape = [64]))

# 2번째 컨볼루션 연산을 통해 14x14x32 => 14x14x64
C3 = tf.nn.conv2d(A2, F3, strides = [1,1,1,1], padding = 'SAME')

Z3 = tf.nn.relu(C3+b3)

# 2번째 max pooling을 통해 14x14x64 => 7x7x64
A3 = P3 = tf.nn.max_pool(Z3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [7]:
# 3번째 컨볼루션 층
# 3x3크기를 가지는 128개의 필터를 적용
F4 = tf.Variable(tf.random_normal([3,3,64,128], stddev=0.01))
b4 = tf.Variable(tf.constant(0.1, shape = [128]))

# 3번째 컨볼루션 연산을 통해 7x7x64 => 7x7x128
C4 = tf.nn.conv2d(A3, F4, strides = [1,1,1,1], padding = 'SAME')

Z4 = tf.nn.relu(C4+b4)

# 3번째 max pooling을 통해 7x7x128 => 4x4x128
A4 = P4 = tf.nn.max_pool(Z4, ksize = [1,2,2,1], strides = [1,2,2,1], padding='SAME')

In [8]:
# 완전연결층
# 4x4 크기를 가진 128개의 activation map을 flatten시킴
A4_flat = P4_flat = tf.reshape(A4, [-1, 128*4*4]) # 1차원 벡터로 만듬 128*4*4 = 2048개 노드를 가지도록 reshpae
# 완전연결층(2048개 노드)과 출력층(10개 노드-one-hot)은 2048x10개 노드가 완전 연결되어 있음

In [9]:
# 출력층
# 선형회귀를 위한 정의
w5 = tf.Variable(tf.random_normal([128*4*4, 10], stddev=0.01))
b5 = tf.Variable(tf.random_normal([10]))

# 출력층 선형회귀 값 z5, 즉 sortmax에 들어가는 입력값
z5 = logits = tf.matmul(A4_flat, w5) + b5
y = A5 = tf.nn.softmax(z5)

In [10]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=z5, labels=T))

optimizer = tf.train.AdamOptimizer(learning_rate) # 성능개선을 위해 AdamOptimizer 사용
train = optimizer.minimize(loss)

In [11]:
# batch_size X 10 데이터에 대해 argmax를 통해 행단위로 비교함
predicted_val = tf.equal(tf.argmax(A5,1), tf.argmax(T,1))
# one-hot인코딩에 의해 출력층 계산값 A5와 정답 T는 batchsize x 10 shape를 가지는 행렬
# argmax의 두번째 인자에 1을 주어 행 단위로 A5와 T를 비교

# batch_size x 10의 True, False를 1 또는 0으로 반환
accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype = tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    start_time = datetime.now()
    for i in range(epochs) :
        total_batch = int(mnist.train.num_examples/batch_size)
        for step in range(total_batch) :
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
            loss_val, _ = sess.run([loss, train], feed_dict = {X: batch_x_data, T : batch_t_data})
            if step % 100 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)
                
    end_time = datetime.now()
    
    print("\nelapsed time = ", end_time - start_time)
    
    # Accuracy 확인
    test_x_data = mnist.test.images # 10000x784
    test_t_data = mnist.test.labels # 10000x10
    
    accuary_val = sess.run(accuracy, feed_dict={X: test_x_data, T : test_t_data})
    print("\nAccuracy = ", accuracy_val)